1. 문서 내용 읽기
2. 문서 분할(쪼개기)
    - 문서를 분할하지 않으면,
        - 토큰 수 초과로 답변을 생성하지 못할 수 있음
        - 문서 길이(input)가 길어서 답변 생성에 시간 소요됨 -> 답변 늦어짐
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있으면, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [ ]:
## 패키지/라이브러리 설치
# %pip install -Uq docx2txt langchain_community langchain-text-splitters langchain_chroma


Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] [-e] <local project path> ...
  c:\Users\1\miniconda3\envs\project\python.exe -m pip install [options] <archive url/path> ...

no such option: --upgrage


In [1]:
from dotenv import load_dotenv
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1. 문서 내용 읽고, 분할 ===========================================================
loader = Docx2txtLoader('law_1.docx')

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size= 1500,
    chunk_overlap= 200,
    length_function= len,
    is_separator_regex= False,
)

document_list = loader.load_and_split(text_splitter=text_splitter)

## 2. 임베딩 -> 벡터 데이터베이스에 저장 =============================================
## 2.1 환경변수 읽어오기
load_dotenv()

## 2.2 임베딩 모델 지정
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

## 2.3 벡터 데이터베이스에 저장
## [방법 1] in memory
database = Chroma.from_documents(
    documents= document_list,
    embedding=embedding,
)

# 3. 질문이 있으면, 벡터 데이터베이스에서 유사도 검색 ================================

## 3.1 사용자 질문
query = '전세사기 피해자 금융지원에 대해 설명해주세요.'

## 3.2 벡터 데이터베이스에서 유사도 검색
retrieved_docs = database.similarity_search(query=query)

# 4. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달 ============================
## 4.1 프롬프트 작성
prompt = '''
[identity]
-당신은 전세사기 피해 법률 전문가입니다.
-[context]를 참고하여 사용자의 질문에 답변해주세요.

[context]
{retrieved_docs}

Question: {query}
'''
## 4.2 프롬프트 변수에 값 설정
formatted_prompt = prompt.format(retrieved_docs=retrieved_docs, query=query)

## 4.3 LLM 모델 생성(ChatOpenAI 인스턴스 생성)
llm = ChatOpenAI(model = 'gpt-4o',)

## 4.4 LLM 모델에 질문과 검색된 문서를 보냄
ai_message = llm.invoke(formatted_prompt)
ai_message

AIMessage(content='전세사기 피해자에 대한 금융지원은 여러 측면에서 이루어집니다. 먼저, 국가 및 지방자치단체는 긴박한 주거안정을 보호하기 위해 필요한 자금을 전세사기 피해자들에게 융자하거나 그 밖의 필요한 지원을 할 수 있습니다. 이 경우, 주거안정을 위한 자금은 주로 「주택도시기금법」에 의한 주택도시기금에서 지원됩니다.\n\n또한, 전세사기 피해자가 「주택임대차보호법」 제8조에 따른 우선변제를 받지 못해 시급한 지원이 필요하다고 인정되는 경우, 주택도시기금에서 임대차에 필요한 자금을 융자 받을 수 있습니다.\n\n한편, 전세사기 피해자의 보증금이 모두 변제되지 않은 경우, 「신용정보의 이용 및 보호에 관한 법률」에 따라 금융회사 등이 이들의 전세 관련 대출에 대한 채무 불이행 및 대위변제의 등록을 유예하거나 삭제할 수 있습니다. 이는 전세사기 피해자가 신용정보에 불리한 영향을 받지 않도록 돕기 위한 조치입니다. \n\n마지막으로, 전세사기 피해자 및 특정 임차인에 대해서는 「한국주택금융공사법」에 따른 주택저당채권 및 준주택 중 주거목적으로 사용되는 오피스텔도 포함하는 등의 금융지원을 제공하고 있습니다. 이러한 지원들은 전세사기 피해자들이 겪는 경제적 부담을 완화하는 데 도움을 주고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 349, 'prompt_tokens': 4166, 'total_tokens': 4515, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-